In [1]:
import auxiliary as aux

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from lightgbm import LGBMRegressor

## best approach

In [21]:
# Kaggle: 0.91069
predicted = aux.ImputeHelper(
    (aux.ml_impute, aux.subcol[4], {'estimator': LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=1500, num_leaves=29), 
                                    'max_fill_nan_count': np.inf, 'max_train_nan_count': np.inf}),
    (aux.simplestat, aux.subcol[1] + aux.subcol[3], SimpleImputer(strategy='mean')),
).run(aux.data, inherit=True)

100%|██████████| 15/15 [23:55<00:00, 95.71s/it] 



ML Imputer avg. score: 0.5946655547740444


100%|██████████| 40/40 [00:00<00:00, 41.11it/s]


SimpleStat avg. score: 0.9655820744615067


In [ ]:
aux.calc_score(aux.data, predicted)

## tests

In [14]:
# check various parameters of NaN count
predicted = aux.ImputeHelper(
    (aux.ml_impute, aux.subcol[4], {'estimator': LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=1500, num_leaves=29), 
                                    'max_fill_nan_count': np.inf, 'max_train_nan_count': np.inf}),
).run(aux.data, inherit=True)

100%|██████████| 15/15 [20:00<00:00, 80.01s/it]



ML Imputer avg. score: 0.5946655547740433


In [ ]:
predicted = aux.ImputeHelper(
    (aux.ml_impute, aux.subcol[4], {'estimator': LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=1500, num_leaves=29), 
                                    'max_fill_nan_count': np.inf, 'max_train_nan_count': 0}),
).run(aux.data, inherit=True)

In [23]:
predicted = aux.ImputeHelper(
    (aux.ml_impute, aux.subcol[4], {'estimator': LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=1500, num_leaves=29), 
                                    'max_fill_nan_count': 1, 'max_train_nan_count': 0}),
    (aux.simplestat, aux.subcol[4], SimpleImputer(strategy='mean')),
).run(aux.data, inherit=True)

100%|██████████| 15/15 [20:14<00:00, 80.95s/it]



ML Imputer avg. score: 0.42641156775337585


100%|██████████| 15/15 [00:00<00:00, 43.49it/s]


SimpleStat avg. score: 1.903149533816095


In [24]:
# CHECK THIS TOO
predicted = aux.ImputeHelper(
    (aux.ml_impute, aux.subcol[4], {'estimator': LGBMRegressor(random_state=7, n_jobs=-1, n_estimators=1500, num_leaves=29), 
                                    'max_fill_nan_count': 2, 'max_train_nan_count': 0}),
    (aux.simplestat, aux.subcol[4], SimpleImputer(strategy='mean')),
).run(aux.data, inherit=True)

## submission

In [22]:
aux.save_submission(predicted)

Still contain NaN: False


,row-col,value
0,0-F_1_14,-0.000905
1,0-F_3_23,0.000365
2,1-F_3_24,-0.000817
3,2-F_1_2,0.000551
4,2-F_4_2,0.232231


In [ ]:
#